# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,summary,description,citation,url_slug,paper_url,clink,pubtype
0,2014-04-11,Assessment of Genetic Diversity in Sunflower (...,International Journal of Agriculture and Food ...,The present investigation was carried out at t...,The present investigation was carried out at t...,"<b>Shamshad, M.</b>, Dhillon, S. K., Tyagi, V....",Genetic-Diversity-in-Sunflower,/files/research/Shamshad_ijafst.pdf,httpx//www:ripublication:com/ ijafst:htm,journal
1,2014-07-11,Magnitude of combining ability for seed yield ...,National Symposium on Crop Improvement for Inc...,The additive components of genetic variance pl...,The additive components of genetic variance pl...,"<b>Shamshad, M.</b>, Dhillon, S. K. (2014). ""M...",Combinig-Ability-sunflower,/files/research/Magnitude_combiningAblity_sunf...,httpx//www:ripublication:com/ ijafst:htm,journal
2,2016-05-18,Heterosis for oil content and oil quality in s...,Current Advances in Agricultural Sciences,The present study was aimed to estimate hetero...,The present study was aimed to estimate hetero...,"<b>Shamshad, M.</b>, Dhillon, S. K., & kaur, G...",Heterosis-for-oil-and-oil-content,/files/research/Heterosis_Oil_content.pdf,https://doi.org/ 10.5958/2394-4471.2016.00010.1,journal
3,2016-01-12,Morphological and molecular genetic diversity ...,Electronic Journal of Plant Breeding,The knowledge of genetic diversity among paren...,The knowledge of genetic diversity among paren...,"<b>Shamshad, M.</b> & Dhillon, S. K. (2016). ""...",Morphological-and-Genetic-diversity-in-sunflower,/files/research/Morphological.pdf,https://doi.org/10.5958/0975-928X.2016.00169.1,journal
4,2022-03-31,RNA-Seq-Based Transcriptomics Study to Investi...,Frontier in Genetics,High NUE (nitrogen use efficiency) has great p...,High NUE (nitrogen use efficiency) has great p...,"Kaur, S., <b>Shamshad, M.</b>, Jindal, S., Kau...","RNA-Seq-NUE,-differential-gene-expression",/files/research/RNA-seq.pdf,https://doi.org/10.3389/fgene.2022.853910,journal
5,2022-01-22,Uncovering the Iranian wheat landraces for sal...,Journal of Cereal Research Communications,Soil salinity is a major global environmental ...,Soil salinity is a major global environmental ...,"Kaur, S., Suhalia, A., Sarlach, R. S., <b>Sham...",Salinity-lranian,/files/research/Irianian_Wheat_Salinity.pdf,https://doi.org/10.1007/s42976-022-00245-6,journal
6,2022-02-19,Salt stress genotypic response: Relative toler...,Agricultural Research Journal,The commercial wheat genotypes released for cu...,The commercial wheat genotypes released for cu...,"Sharma, A., Ankita, <b>Shamshad, M.</b>, Kaur,...",PAU-Salinity,https://doi.org/10.5958/2395-146X.2021.00138.0,https://doi.org/10.5958/2395-146X.2021.00138.1,journal
7,2022-04-06,"Pyramiding of genes for grain protein content,...",Molecular Breeding,"Improvement of grain protein content (GPC), lo...","Improvement of grain protein content (GPC), lo...","Gupta, P. K., Balyan, H. S., Chhuneja, P., Jai...",Gpc-Gy-Pyramiding-genes,/files/research/Pyramiding-Gpc-Yield.pdf,https://doi.org/10.1007/s11032-022-01277-w,journal
8,2022-05-05,Unravelling consensus genomic regions associat...,Planta,A meta-analysis of quantitative trait loci (QT...,A meta-analysis of quantitative trait loci (QT...,"Gudi, S., Saini, D. K., Singh, G., Halladakeri...",Meta-Analysis-quality-trait,/files/research/Meta-Analysis-Gpc.pdf,https://doi.org/10.1007/s00425-022-03904-4,journal
9,2022-05-17,In silico characterization of <i>Thinopyrum el...,Journal of Cereal Research Communications,Current global scenario demands agricultural p...,Current global scenario demands agricultural p...,"Padhy, A. K., Kaur, P., Singh, B., Kaur, R., B...",Carotenoid-biofortification,/files/research/LTP_NUE_GWAS.pdf,https://doi.org/10.1007/s42976-022-00279-w,journal


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    ## YAML variables
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications\npermalink: /publication/""" + html_filename
    md += "\nexcerpt: '"  + html_escape(item.summary) + "'"
    md += "\ndate: " + str(item.pub_date) 
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    md +="\nclink: '" + html_escape(item.clink) + "'"
    md += "\npurl: '" + item.paper_url + "'"
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\npubtype: '" + html_escape(item.pubtype) + "'"
    md += "\n---"
    md += "\n\n<a href='" + item.paper_url + "'>Download PDF here</a>\n" 
    md += "\nAbstract: " + html_escape(item.description) + "\n"
    md += "\nRecommended citation: " + item.citation

    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [9]:
cd ../_publications/

C:\Users\M Shamshad\Documents\website\_publications


In [6]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!type  2022-05-17-Carotenoid biofortification.md

The system cannot find the file specified.
Error occurred while processing: 2022-05-17-Carotenoid.
The system cannot find the file specified.
Error occurred while processing: biofortification.md.
